# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [140]:
# import libraries
import pandas as pd
import numpy as np
from sqlalchemy import create_engine

from nltk.stem import WordNetLemmatizer
from nltk.tokenize import RegexpTokenizer

from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import LinearSVC
from sklearn.linear_model import SGDClassifier, LogisticRegression
from sklearn.metrics import classification_report



In [103]:
# load data from database
engine = create_engine('sqlite:///data/DisasterResponse.db')
df = pd.read_sql_table('DisasterResponse', engine)  
X = df["message"]
Y = df.iloc[:,-36:]

### 2. Write a tokenization function to process your text data

In [106]:
def tokenize(text):
    tokenizer = RegexpTokenizer(r'\w+')
    lemmatizer = WordNetLemmatizer()

    tokens = tokenizer.tokenize(text)

    clean_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)

    return clean_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [130]:
'''
This model uses the following steps recommended in 
https://towardsdatascience.com/machine-learning-nlp-text-classification-using-scikit-learn-python-and-nltk-c52b92a7c73a 
for multi-output classification task:
1. CountVectorizer()
2. TfidfTransformer()
3. Classifier

'''

pipeline =  Pipeline([
                ('vect', CountVectorizer(tokenizer=tokenize)),
                ('tfidf', TfidfTransformer()),
                ('clf', MultiOutputClassifier(DecisionTreeClassifier()))
                ])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [131]:
# Split data into train and test
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=21)

# Train pipeline
pipeline.fit(X_train, y_train)

Pipeline(steps=[('vect',
                 CountVectorizer(tokenizer=<function tokenize at 0x0000021E89B2B490>)),
                ('tfidf', TfidfTransformer()),
                ('clf',
                 MultiOutputClassifier(estimator=DecisionTreeClassifier()))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [133]:
# Infer test data with trained model
y_pred = pipeline.predict(X_test)
y_pred = pd.DataFrame(y_pred, columns=y_test.columns)

print(classification_report(y_test.values, y_pred.values, target_names=y_test.columns.values))

                        precision    recall  f1-score   support

               related       0.84      0.84      0.84      4037
               request       0.55      0.56      0.56       877
                 offer       0.05      0.05      0.05        21
           aid_related       0.63      0.63      0.63      2158
          medical_help       0.37      0.30      0.33       432
      medical_products       0.38      0.39      0.39       252
     search_and_rescue       0.27      0.21      0.24       171
              security       0.06      0.05      0.05       106
              military       0.38      0.37      0.37       172
           child_alone       0.00      0.00      0.00         0
                 water       0.66      0.65      0.66       320
                  food       0.70      0.72      0.71       574
               shelter       0.59      0.59      0.59       442
              clothing       0.57      0.46      0.51        84
                 money       0.36      

d:\Anaconda3\envs\nano\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
d:\Anaconda3\envs\nano\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
d:\Anaconda3\envs\nano\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
d:\Anaconda3\envs\nano\lib\site-packages\sklearn\metrics\_classificat

In [139]:
# Alternative:
for col in y_test.columns:
    print(col)
    print(classification_report(y_test.loc[:,col].values, y_pred.loc[:,col].values))

related
              precision    recall  f1-score   support

           0       0.46      0.47      0.46      1207
           1       0.84      0.84      0.84      4037

    accuracy                           0.75      5244
   macro avg       0.65      0.65      0.65      5244
weighted avg       0.75      0.75      0.75      5244

request
              precision    recall  f1-score   support

           0       0.91      0.91      0.91      4367
           1       0.55      0.56      0.56       877

    accuracy                           0.85      5244
   macro avg       0.73      0.73      0.73      5244
weighted avg       0.85      0.85      0.85      5244

offer
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      5223
           1       0.05      0.05      0.05        21

    accuracy                           0.99      5244
   macro avg       0.52      0.52      0.52      5244
weighted avg       0.99      0.99      0.99      524

### 6. Improve your model
Use grid search to find better parameters. 

In [149]:
# List all parameters from pipeline 
pipeline.get_params()

{'memory': None,
 'steps': [('vect',
   CountVectorizer(tokenizer=<function tokenize at 0x0000021E89B2B490>)),
  ('tfidf', TfidfTransformer()),
  ('clf', MultiOutputClassifier(estimator=DecisionTreeClassifier()))],
 'verbose': False,
 'vect': CountVectorizer(tokenizer=<function tokenize at 0x0000021E89B2B490>),
 'tfidf': TfidfTransformer(),
 'clf': MultiOutputClassifier(estimator=DecisionTreeClassifier()),
 'vect__analyzer': 'word',
 'vect__binary': False,
 'vect__decode_error': 'strict',
 'vect__dtype': numpy.int64,
 'vect__encoding': 'utf-8',
 'vect__input': 'content',
 'vect__lowercase': True,
 'vect__max_df': 1.0,
 'vect__max_features': None,
 'vect__min_df': 1,
 'vect__ngram_range': (1, 1),
 'vect__preprocessor': None,
 'vect__stop_words': None,
 'vect__strip_accents': None,
 'vect__token_pattern': '(?u)\\b\\w\\w+\\b',
 'vect__tokenizer': <function __main__.tokenize(text)>,
 'vect__vocabulary': None,
 'tfidf__norm': 'l2',
 'tfidf__smooth_idf': True,
 'tfidf__sublinear_tf': False,


In [ ]:
'''
Crucial parameters https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html 
to avoid fully grown and unpruned trees in decision tree classifier algorithms:
- clf__estimator__max_depth
-clf__estimator__min_samples_leaf
'''
parameters = 

cv = 

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.